## Metaclasses and allocation

### Instance creation
* when creating an object, the \_\_init\_\_ method will be invoke. Before this method is invoked, the object has already been created, and passed to this method as 'self'
* during initialization process, the attributes of the object are assigned/modified by object.\_\_setattr\_\_ method
* \_\_init\_\_ method doesn't return anything. It just mutate what the self object was given
* since \_\_init\_\_ method doesn't create new object, which method does this?
  + \_\_new\_\_
  + usually \_\_new\_\_ is inherited from Object. This method allocates object which is then passed to \_\_init\_\_ as self
  + \_\_new\_\_ is an implicit static method having cls as its first argument
  + cls is the class of the object which will be allocated. In the code example, this will be ChessCoordinate class object
  + this method must use the cls as the first argument with other arguments forwarded from constructor calls, to create an object and and return the object
    + example code shows that the arguments provided when instantialte the ChessCoordinate are received from \_\_new\_\_()
    + the \_\_new\_\_() method only use cls to create the object with an id, and then return the object
    + the object and other arguments of object instantiation ('d', and 4) are forwarded to \_\_init\_\_ as file and rank, respectively
  

In [16]:
class ChessCoordinate:
    
    def __new__(cls, *args, **kwargs):
        print(f"cls = {cls.__name__}")
        print(f"args = {args!r}")
        print(f"kwargs = {kwargs!r}")
        obj = object.__new__(cls)
        print(f"id(obj) = {id(obj)}")
        return obj
    
    def __init__(self, file, rank):
        
        print(f"id(self) = {id(self)}")
        
        if len(file) != 1:
            raise ValueError(
                f"{type(self).__name__} component file {file!r} "
                f"does not have a length of one."
            )
            
        if file not in "abcdefgh":
            raise ValueError(
                f"{type(self).__name__} component file {file!r} "
                f"is out of range."
            )
        
        self._file = file
        self._rank = rank
        
    @property
    def file(self):
        return self._file

    @property
    def rank(self):
        return self._rank

    def __repr__(self):
        return f"{type(self).__name__}(file={self.file}, rank={self.rank})"

    def __str__(self):
        return f"{self.file}{self.rank}"       

In [17]:
white_queen = ChessCoordinate("d", 4)
print(white_queen)
        

cls = ChessCoordinate
args = ('d', 4)
kwargs = {}
id(obj) = 1854128552736
id(self) = 1854128552736
d4


In [23]:
class ChessCoordinate:
    
    def __new__(cls, *args, **kwargs):        
        obj = object.__new__(cls)        
        return obj
    
    def __init__(self, file, rank):        
              
        if len(file) != 1:
            raise ValueError(
                f"{type(self).__name__} component file {file!r} "
                f"does not have a length of one."
            )
            
        if file not in "abcdefgh":
            raise ValueError(
                f"{type(self).__name__} component file {file!r} "
                f"is out of range."
            )
        
        self._file = file
        self._rank = rank
        
    @property
    def file(self):
        return self._file

    @property
    def rank(self):
        return self._rank

    def __repr__(self):
        return f"{type(self).__name__}(file={self.file}, rank={self.rank})"

    def __str__(self):
        return f"{self.file}{self.rank}"       

In [24]:
# code for measuring memory usage
import tracemalloc
tracemalloc.start()
white_queens = [ ChessCoordinate("d", 4) for _ in range(10000)]
_, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
peak_kb = peak / 10 **3
print(f"{peak_kb:.0f} KB")

1616 KB


### Customizing allocation
#### Interning
* re-using objects of equal value on-demand instead of creating new objects
* we can interning objects by overriding \_\_new\_\_
* when we create a large numbers of object, the memory usage is linearly increased
* for our use case, since there are only 64 positions on the chessboard, we only need 64 or 32 boards to record all the positions on the board.
+ to implement the interning, we move the code from \_\_init\_\_ to \_\_new\_\_ and put arguments of instantiation to \_\_new\_\_
  + we change the self to cls before object creation, and after ChessCoordinate object is created, assign the \_file and \_rank to object, rather than self in \_\_new\_\_
  + we created a dictionary in the class definition to store the ChessCoordinate instance using its file and rank combination as the key. We only create new ChessCoordinate instances corresponding to a new key and store the created object to the \_interned dictionary. Otherwise, we just retrieve the object from the dictionary and return it 
  + we then delete the \_\_init\_\_ method
* intering can reduce the memory usage by reducing the number of instances created
  + it can only used for immutable value types

In [25]:
class ChessCoordinate:
    
    _interned = {}
    
    def __new__(cls, file, rank):
        
        if len(file) != 1:
            raise ValueError(
                f"{cls.__name__} component file {file!r} "
                f"does not have a length of one."
            )
            
        if file not in "abcdefgh":
            raise ValueError(
                f"{cls.__name__} component file {file!r} "
                f"is out of range."
            )
        
        key = (file, rank)
        if key not in cls._interned:
            obj = object.__new__(cls) 
            obj._file = file
            obj._rank = rank
            cls._interned[key] = obj
        
        return cls._interned[key]
    
            
    @property
    def file(self):
        return self._file

    @property
    def rank(self):
        return self._rank

    def __repr__(self):
        return f"{type(self).__name__}(file={self.file}, rank={self.rank})"

    def __str__(self):
        return f"{self.file}{self.rank}"       

In [26]:
# code for measuring memory usage
import tracemalloc
tracemalloc.start()
white_queens = [ ChessCoordinate("d", 4) for _ in range(10000)]
_, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
peak_kb = peak / 10 **3
print(f"{peak_kb:.0f} KB")

98 KB


#### Summary of instance allocation and creation
* the special method \_\_new\_\_ allocates and return new instances
* the new instance is passed to \_\_init\_\_ as self
* the ultimat allocator is object.\_\_new\_\_(cls) that is responsible for allocating all instances. It must be passed the class of object to be allocated
* we can override \_\_new\_\_ in our own classes to customize allocation
  + the \_\_new\_\_(cls) method is implicitly static, which accept the class of the new instance as its first argument, and does not require classmethod or staticmethod decorators
* interning reuses existing objects of equal value (in our code example, the key of the instance represented by file and rank) to save memory usage 
  + this can be useful when certain values of immutable value types are very common, or when the domain of values is small and finite 

### Metaclasses and Class Creation

#### class creation and Metaclasses
* metaclasses
  + class of classes
  + type of any class object in Python is its metaclass, and the default metaclass is type, including type class itself
* when we define a class, we implicitly use object as the default base class, and type as the default metaclass. Specifically,

```python
class Widget:
    pass
``` 
can be explicitly expressed as 

```python
class Widget(object, metaclass=type):
    pass
```
* class definition process
  + when we define a class using class Widget,
    + the new class's metaclass creates an empty namespace dictionary
    + python runtime extracts the content of the class definition block and populates the dictionary
    + python runtime then hand it back to metaclass 
    + metaclass convert it into the class object that bound to class name (allocates class object)
  + in the following code, the name, bases, and namespace arguments contain information collected during execution of class definition, class attributes and method definitions inside the class block
  + by providing our own metaclass, we can customize these behaviors  
  + to use python code to describe the process:
  ```python
    class Widget:
        pass
    
    # the following are what happens:
    
    name = 'Widget'      # class name is Widget
    metaclass = type     # metaclass is the default, type
    bases = ()           # no base class other than the implicit object
    kwargs = {}          # no keyword args
    
    # metaclass create a new namespace, which behaviors like a dictionary
    namespace = metaclass.__prepare__(name, bases, **kwargs)
    # after this call, python runtime populates this namespace dictionary with entries based on
    # the contents of the class block in source code
    
    # metaclass's __new__ method is called to allocate the object
    Widget = metaclass.__new__(metaclass, name, bases, namespace, **kwargs)
    
    # then metaclass's __init__ method is called to initialize the class object
    metaclass.__init__(Widget, name, bases, namespace, **kwargs)
    
    ```    

In [27]:
class Widget:
    pass
w = Widget()

# metaclass of an instance is its class
print(w.__class__)
print(w.__class__.__class__)
print(w.__class__.__class__.__class__)

<class '__main__.Widget'>
<class 'type'>
<class 'type'>


#### code demo of the process
* the following code shows the entire process by defining our own metaclass: TracingMeta
* TracingMeta just print out the cls, name, namespace, bases, keyword args values during the new object creation process
* it inherits the default metaclass, type
* all the methods in TracingMeta just called the super(), which are the methods executed by type metaclass
* note that \_\_new\_\_ is an implicit class method, and we need to decorate \_\_prepare\_\_ by @classmethod

In [30]:
class TracingMeta(type):
    
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        """
        mcs:      metaclass itself, similar to 'self' in a normal class method signature
        name:     name of the metaclass
        bases:    a tuple of base classes, the ultimate object class is implicite
        **kwargs: an empty dictionary
        """
        print("TracingMeta.__prepare__(name, bases, **kwargs)")
        print(f" {mcs = }")
        print(f" {name = }")
        print(f" {bases = }")
        print(f" {kwargs = }")
        
        # returns a mapping type, here an empty dictionary as a namespace
        # assciated with the nascent class
        namespace = super().__prepare__(name, bases)
        print(f"-> {namespace = }")
        print()
        return namespace
    
    # create new class object
    def __new__(mcs, name, bases, namespace, **kwargs):
        """
        namespace: namespace mapping returned from __prepare__
        """
        print("TracingMeta.__new__(mcs, name, bases, namespace)")
        print(f" {mcs = }")
        print(f" {name = }")
        print(f" {bases = }")
        
        # python runtime populated this dictionary with several entries
        # as it processed the class definition of Widget class.
        # including class attributes and methods defined in the class
        # it also adds module, which is the name of the file that contains this class
        # and fully qualified name, which is a gobal class name since it is in bulit-in
        print(f" {namespace = }")
        print(f" {kwargs = }")
        
        # create a new Widget class object
        # any changes we want to make to the namespace object must be made before this call
        # to change the contents of the class namespace after this call,the class object
        # must be manipulated directly
        cls = super().__new__(mcs, name, bases, namespace)
        print(f"-> {cls = }")
        print()
        return cls
    
    # this method configure the class object
    # we could modify namespace by the method, before the class object is instantiated
    # we could modify the sequence of base classes, and allocate a different class entirely to cls
    def __init__(cls, name, bases, namespace, **kwargs):
        """
        this is an instance method of the metaclass. both __new__ and __prepare__ are class methods
        cls: Widget class object (one level less than TracingMeta class, the same way 'self' less than cls)
        """
        print("TracingMeta.__init__(cls, name, bases, namespace)")
        print(f" {cls = }")
        print(f" {name = }")
        print(f" {bases = }")
        print(f" {namespace = }")
        print(f" {kwargs = }")
        super().__init__(name, bases, namespace)
        print(f"-> {cls = }")
        print()  
    

In [31]:
class Widget(metaclass=TracingMeta):
    the_answer = 42
    
    def action(self, message):
        print(message)

TracingMeta.__prepare__(name, bases, **kwargs)
 mcs = <class '__main__.TracingMeta'>
 name = 'Widget'
 bases = ()
 kwargs = {}
-> namespace = {}

TracingMeta.__new__(mcs, name, bases, namespace)
 mcs = <class '__main__.TracingMeta'>
 name = 'Widget'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'Widget', 'the_answer': 42, 'action': <function Widget.action at 0x000001AFB3616EF0>}
 kwargs = {}
-> cls = <class '__main__.Widget'>

TracingMeta.__init__(cls, name, bases, namespace)
 cls = <class '__main__.Widget'>
 name = 'Widget'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'Widget', 'the_answer': 42, 'action': <function Widget.action at 0x000001AFB3616EF0>}
 kwargs = {}
-> cls = <class '__main__.Widget'>



#### Which methods we should override
* \_\_prepare\_\_:
  + customize the type or initial value of the namespace mapping
  + default is to generate a regular empty dictionary for the namespace object
  + you only override this method if you need another mapping type
* usually it will be necessary to override either \_\_new\_\_ or \_\_init\_\_
  + only __new__ can make decisions before the new class is allocated
  + can configure the class object so that metaclasses are more composable

### Metaclass details

#### Metaclass keyword arguments
* any keyword arguments above the metaclass argument in the class definition will be fowarded to the 3 metaclass methods
```python
# more, keyword, and args will be forwarded to the 3 metaclass methods
class Widget(object, metaclass=type, more=1, keyword=2, args=3):
    pass
```

* using the keyward args can make metaclass as a class factory

In [32]:
class Reticulator(metaclass=TracingMeta, tension=496):
    cubic = True
    def reticulate(self, spline):
        print(spline)

TracingMeta.__prepare__(name, bases, **kwargs)
 mcs = <class '__main__.TracingMeta'>
 name = 'Reticulator'
 bases = ()
 kwargs = {'tension': 496}
-> namespace = {}

TracingMeta.__new__(mcs, name, bases, namespace)
 mcs = <class '__main__.TracingMeta'>
 name = 'Reticulator'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'Reticulator', 'cubic': True, 'reticulate': <function Reticulator.reticulate at 0x000001AFB36167A0>}
 kwargs = {'tension': 496}
-> cls = <class '__main__.Reticulator'>

TracingMeta.__init__(cls, name, bases, namespace)
 cls = <class '__main__.Reticulator'>
 name = 'Reticulator'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'Reticulator', 'cubic': True, 'reticulate': <function Reticulator.reticulate at 0x000001AFB36167A0>}
 kwargs = {'tension': 496}
-> cls = <class '__main__.Reticulator'>



In [43]:
class EntriesMeta(type):
    
    def __new__(mcs, name, bases, namespace, num_entries):
        print("EntriesMeta.__new__(mcs, name, bases, namespace, **kwargs)")
        print(f" {num_entries = }")
        namespace.update({chr(i): i for i in range(ord('a'), ord('a') + num_entries)})
        cls = super().__new__(mcs, name, bases, namespace)
        return cls        

In [44]:
class AtoZ(metaclass=EntriesMeta, num_entries=26):
    pass

EntriesMeta.__new__(mcs, name, bases, namespace, **kwargs)
 num_entries = 26


In [45]:
from pprint import pprint
pprint(dir(AtoZ))
AtoZ.f

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']


102

#### Metaclass method visibility
* other methods except for the three methods above in metaclass definition can be access as regular methods from the class objects created by class_name.method_name. These methods are called metamethods
* we can't access metamethods from the instances of the class, as shown in the code example
* metamethods are rarely used, except for \_\_call\_\_() method
* class methods defined in the class and its base classes will take the precedence over the class methods defined in its metaclass and metaclass's base class

In [46]:
class TracingMeta(type):
    
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        """
        mcs:      metaclass itself, similar to 'self' in a normal class method signature
        name:     name of the metaclass
        bases:    a tuple of base classes, the ultimate object class is implicite
        **kwargs: an empty dictionary
        """
        print("TracingMeta.__prepare__(name, bases, **kwargs)")
        print(f" {mcs = }")
        print(f" {name = }")
        print(f" {bases = }")
        print(f" {kwargs = }")
        
        # returns a mapping type, here an empty dictionary as a namespace
        # assciated with the nascent class
        namespace = super().__prepare__(name, bases)
        print(f"-> {namespace = }")
        print()
        return namespace
    
    # create new class object
    def __new__(mcs, name, bases, namespace, **kwargs):
        """
        namespace: namespace mapping returned from __prepare__
        """
        print("TracingMeta.__new__(mcs, name, bases, namespace)")
        print(f" {mcs = }")
        print(f" {name = }")
        print(f" {bases = }")
        
        # python runtime populated this dictionary with several entries
        # as it processed the class definition of Widget class.
        # including class attributes and methods defined in the class
        # it also adds module, which is the name of the file that contains this class
        # and fully qualified name, which is a gobal class name since it is in bulit-in
        print(f" {namespace = }")
        print(f" {kwargs = }")
        
        # create a new Widget class object
        # any changes we want to make to the namespace object must be made before this call
        # to change the contents of the class namespace after this call,the class object
        # must be manipulated directly
        cls = super().__new__(mcs, name, bases, namespace)
        print(f"-> {cls = }")
        print()
        return cls
    
    # this method configure the class object
    # we could modify namespace by the method, before the class object is instantiated
    # we could modify the sequence of base classes, and allocate a different class entirely to cls
    def __init__(cls, name, bases, namespace, **kwargs):
        """
        this is an instance method of the metaclass. both __new__ and __prepare__ are class methods
        cls: Widget class object (one level less than TracingMeta class, the same way 'self' less than cls)
        """
        print("TracingMeta.__init__(cls, name, bases, namespace)")
        print(f" {cls = }")
        print(f" {name = }")
        print(f" {bases = }")
        print(f" {namespace = }")
        print(f" {kwargs = }")
        super().__init__(name, bases, namespace)
        print(f"-> {cls = }")
        print()  
    
    def metamethod(cls):
        print("TracingMeta.metamethod(cls)")
        print(f" {cls = }")

In [47]:
class Widget(metaclass=TracingMeta):
    pass

TracingMeta.__prepare__(name, bases, **kwargs)
 mcs = <class '__main__.TracingMeta'>
 name = 'Widget'
 bases = ()
 kwargs = {}
-> namespace = {}

TracingMeta.__new__(mcs, name, bases, namespace)
 mcs = <class '__main__.TracingMeta'>
 name = 'Widget'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'Widget'}
 kwargs = {}
-> cls = <class '__main__.Widget'>

TracingMeta.__init__(cls, name, bases, namespace)
 cls = <class '__main__.Widget'>
 name = 'Widget'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'Widget'}
 kwargs = {}
-> cls = <class '__main__.Widget'>



In [48]:
# we can access regular class methods from class objects
Widget.metamethod()

TracingMeta.metamethod(cls)
 cls = <class '__main__.Widget'>


In [49]:
# but we can't access these regular class mehods from the class instance objects
w = Widget()
w.metamethod()

AttributeError: 'Widget' object has no attribute 'metamethod'

#### Instance constructor
* when we create new class instance objects, we call the class constructor, which will call \_\_new\_\_ followed by \_\_init\_\_ in the class. This is the responsiblity of \_\_call\_\_ of metaclass
* assuming class Widget's metaclass is type, when we call w = Widget(), the \_\_call\_\_() method in type class will call the \_\_new\_\_(cls) method and \_\_init\_\_(). One implementation of \_\_call\_\_ is the following:

```python
class Widget(object, metaclass=type):
    
    def __new__(cls, *args, **kwargs):
        return type.__new__(cls)
    
    def __init__(self, *args, **kwargs):
        pass
```

```python
class type:
    def __call__(cls, *args, **kwargs):
        obj = cls.__new__(*args, **kwargs)
        
        obj.__init__(*args, **kwargs)
        return obj
```    
* note that the \_\_call\_\_() method is an instance method in the metaclass.
  + it obtain the cls (similar to self in class) in the metaclss after the \_\_new\_\_ method in the metaclass creates cls
  + it then use cls t call \_\_new\_\_() in the class, obtain the class instance object
  + then call the object's \_\_init\_\_() to configure the instance object 
  + see core python notes page 4 for the graph representation
* the following code implemented a class of TracingClass and its metaclass TracingMeta to show the whole process of class instance object creation:
  + once we load the class definitions for TracingClass and TracingMeta, \_\_new\_\_() and \_\_init\_\_() methods in TracingMeta runs, and creates the class object for TracingClass cls
  + we then instantiate an instance of TracingClass, t by
    t = TracingClass(42, keyword="clef")
    + we can see:
    TracingMeta.\_\_call\_\_(cls, \*args, \*\*kwargs)
is called, with args = (42,) and kwargs ={'keyword="clef"} passed to the method       

    + then the \_\_call\_\_() function invoked 
    
    ```python
    TracingClass.__new__(cls, *args, **kwargs)    
    TracingClass.__init__(self, *args, **kwargs)
    ```
    
    + when \_\_call\_\_() returned, it returned the newly created object, obj                    
    + compare the addresses of the objs obtained from the returned value of TracingMeta.\_\_call\_\_, TracingClass.\_\_new\_\_, and TracingClass.\_\_init\_\_, we can see they all referred to the same object


In [52]:
class TracingMeta(type):
    
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        """
        mcs:      metaclass itself, similar to 'self' in a normal class method signature
        name:     name of the metaclass
        bases:    a tuple of base classes, the ultimate object class is implicite
        **kwargs: an empty dictionary
        """
        print("TracingMeta.__prepare__(name, bases, **kwargs)")
        print(f" {mcs = }")
        print(f" {name = }")
        print(f" {bases = }")
        print(f" {kwargs = }")
        
        # returns a mapping type, here an empty dictionary as a namespace
        # assciated with the nascent class
        namespace = super().__prepare__(name, bases)
        print(f"-> {namespace = }")
        print()
        return namespace
    
    # create new class object
    def __new__(mcs, name, bases, namespace, **kwargs):
        """
        namespace: namespace mapping returned from __prepare__
        """
        print("TracingMeta.__new__(mcs, name, bases, namespace)")
        print(f" {mcs = }")
        print(f" {name = }")
        print(f" {bases = }")
        
        # python runtime populated this dictionary with several entries
        # as it processed the class definition of Widget class.
        # including class attributes and methods defined in the class
        # it also adds module, which is the name of the file that contains this class
        # and fully qualified name, which is a gobal class name since it is in bulit-in
        print(f" {namespace = }")
        print(f" {kwargs = }")
        
        # create a new Widget class object
        # any changes we want to make to the namespace object must be made before this call
        # to change the contents of the class namespace after this call,the class object
        # must be manipulated directly
        cls = super().__new__(mcs, name, bases, namespace)
        print(f"-> {cls = }")
        print()
        return cls
    
    # this method configure the class object
    # we could modify namespace by the method, before the class object is instantiated
    # we could modify the sequence of base classes, and allocate a different class entirely to cls
    def __init__(cls, name, bases, namespace, **kwargs):
        """
        this is an instance method of the metaclass. both __new__ and __prepare__ are class methods
        cls: Widget class object (one level less than TracingMeta class, the same way 'self' less than cls)
        """
        print("TracingMeta.__init__(cls, name, bases, namespace)")
        print(f" {cls = }")
        print(f" {name = }")
        print(f" {bases = }")
        print(f" {namespace = }")
        print(f" {kwargs = }")
        super().__init__(name, bases, namespace)
        print(f"-> {cls = }")
        print()  
    
    def metamethod(cls):
        print("TracingMeta.metamethod(cls)")
        print(f" {cls = }")
        
        
    def __call__(cls, *args, **kwargs):
        print("TracingMeta.__call__(cls, *args, **kwargs)")
        print(f" {cls = }")
        print(f" {args = }")
        print(f" {kwargs = }")
        print("about to call type.__call__()")
        obj = super().__call__(*args, **kwargs)
        print("returned from type.__call__()")
        print(f"-> {obj = }")
        print()
        return obj
    
class TracingClass(metaclass=TracingMeta):
    
    def __new__(cls, *args, **kwargs):
        print(" TracingClass.__call__(cls, *args, **kwargs)")
        print(f" {cls = }")
        print(f" {args =}")
        print(f" {kwargs = }")
        obj = super().__new__(cls)
        print(f" -> {obj = }")
        print()
        return obj
    
    def __init__(self, *args, **kwargs):
        print("TracingClass.__init__(cls, *args, **kwargs)")
        print(f" {self = }")
        print(f" {args = }")
        print(f" {kwargs = }")
        print()
         

TracingMeta.__prepare__(name, bases, **kwargs)
 mcs = <class '__main__.TracingMeta'>
 name = 'TracingClass'
 bases = ()
 kwargs = {}
-> namespace = {}

TracingMeta.__new__(mcs, name, bases, namespace)
 mcs = <class '__main__.TracingMeta'>
 name = 'TracingClass'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'TracingClass', '__new__': <function TracingClass.__new__ at 0x000001AFB2E70430>, '__init__': <function TracingClass.__init__ at 0x000001AFB2E70B80>, '__classcell__': <cell at 0x000001AFB2CCCBB0: empty>}
 kwargs = {}
-> cls = <class '__main__.TracingClass'>

TracingMeta.__init__(cls, name, bases, namespace)
 cls = <class '__main__.TracingClass'>
 name = 'TracingClass'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'TracingClass', '__new__': <function TracingClass.__new__ at 0x000001AFB2E70430>, '__init__': <function TracingClass.__init__ at 0x000001AFB2E70B80>, '__classcell__': <cell at 0x000001AFB2CCCBB0: TracingMeta object at 0x000001AFAFF

In [53]:
t = TracingClass(42, keyword="clef")

TracingMeta.__call__(cls, *args, **kwargs)
 cls = <class '__main__.TracingClass'>
 args = (42,)
 kwargs = {'keyword': 'clef'}
about to call type.__call__()
 TracingClass.__call__(cls, *args, **kwargs)
 cls = <class '__main__.TracingClass'>
 args =(42,)
 kwargs = {'keyword': 'clef'}
 -> obj = <__main__.TracingClass object at 0x000001AFB2BEB790>

TracingClass.__init__(cls, *args, **kwargs)
 self = <__main__.TracingClass object at 0x000001AFB2BEB790>
 args = (42,)
 kwargs = {'keyword': 'clef'}

returned from type.__call__()
-> obj = <__main__.TracingClass object at 0x000001AFB2BEB790>



#### Phased initialization
* template method pattern
  + define an outline but defer details by delegating the details to other methods, which can potentially be overriden in sub classes.
  + in the following code example, the template \_\_init\_\_ refers to three \_\_int\_\_ methods, each of which prints messages
  + we can override the three methods in its subclasses in any combinations
* the challenge of template pattern is that you need to know it has been used, and which method(s) should be overriden, and which should not 
  + we may inadvertly override the wrong methods, for example \_\_init\_\_, rather than one of the 3 methods inside it, and we may call the separate init methods in the wrong order etc.
  + one solution is to use the metadata class to manage the execution of these init methods
    + the following code example shows how to manage the init methods in a metaclass
    + in this example, we defined \_pre\_init, \_\_init\_\_ and \_post\_init, and then in metaclass.\_\_call\_\_ function, defines the order of executions.
      + this is impossible without using metaclass, since \_\_init\_\_ will always call before other methods if not overriding the sequence in metaclass
  

In [54]:
class PhasedInit:
    
    def __init__(self):
        self._pre_init()
        self._do_init()
        self._post_init()
        
    def _pre_init(self):
        print("Pre-initialization")
        
    def _do_init(self):
        print("Initialization")
        
    def _post_init(self):
        print("Post-initialization")

In [55]:
p = PhasedInit()
print(p)

Pre-initialization
Initialization
Post-initialization


In [56]:
class SubPhasedInit(PhasedInit):
    
    def _do_init(self):
        print("Sub-initialization")

In [57]:
p = SubPhasedInit()
print(p)

Pre-initialization
Sub-initialization
Post-initialization


In [59]:
class PhasedMeta(type):
    
    def __call__(cls, *args, **kwargs):
        obj = cls.__new__(cls, *args, **kwargs)
        obj._pre_init(*args, **kwargs)
        obj.__init__(*args, **kwargs)
        obj._post_init(*args, **kwargs)
        return obj

class PhasedInit(metaclass=PhasedMeta):
    
    def _pre_init(self):
        print("Pre-initialization")
        
    def _init__(self):
        print("Initialization")
        
    def _post_init(self):
        print("Post-initialization")
        
class SubPhasedInit(PhasedInit):
    
    def __init__(self):
        print("Sub-initialization")        

In [60]:
p = SubPhasedInit()
print(p)

Pre-initialization
Sub-initialization
Post-initialization


#### Custom Namespace Dictionaries
* implement a metaclass to detect duplicated methods defined in class
  + we can define a custom dictionary that disallow teh setting of a key more than once as the namespace mapping. Specifically, we need
    + specialize \_\_init\_\_ to establish uniqueness invariant
    + specialize \_\_setitem\_\_ to maintain uniqueness invariant
 + we can improve further by adding the name of the class as a dictionary attribute, and print out the class name in error message. 
   + the class name can be passed to \_\_prepare\_\_ method   

In [65]:
# create a dictionary that only allow to set key once
class OneShotNamespace(dict):
    
    def __init__(self, name, existing=None):
        super().__init__()
        self._name = name
        if existing is not None:
            for k, v in existing.items():
                self[k] = v
                
    def __setitem__(self, key, value):
        if key in self:
            raise TypeError(
                f"Cannot reassign attribute {key!r}"
                f"of class {self._name!r}"
            )
        super().__setitem__(key, value) 
        
# use OneShotDict as the namespace of class
# define this dict obj in metaclass
class ProhibitDuplicatesMeta(type):
    
    @classmethod
    def __prepare__(mcs, name, bases):
        return OneShotNamespace(name)
    
# implement Dodgy class using the metaclass
class Dodgy(metaclass=ProhibitDuplicatesMeta):
    
    def method(self):
        return "first definition"
    
    def method(self):
        return "second definition"    

TypeError: Cannot reassign attribute 'method'of class 'Dodgy'

In [63]:
# test OneShotDict
d = OneShotDict()
d['A'] = 65
d['B'] = 66
d['A'] = 65

KeyError: "Cannot assign to existing key 'A'"

In [ ]:
# create a dictionary that only allow to set key once
class OneShotDict(dict):
    
    def __init__(self, existing=None):
        super().__init__()
        if existing is not None:
            for k, v in existing.items():
                self[k] = v
                
    def __setitem__(self, key, value):
        if key in self:
            raise KeyError(
                f"Cannot assign to existing key {key!r}"
            )
        super().__setitem__(key, value) 
        
# use OneShotDict as the namespace of class
# define this dict obj in metaclass
class ProhibitDuplicatesMeta(type):
    
    @classmethod
    def __prepare__(cls, name, bases):
        return OneShotDict()
    
# implement Dodgy class using the metaclass
class Dodgy(metaclass=ProhibitDuplicatesMeta):
    
    def method(self):
        return "first definition"
    
    def method(self):
        return "second definition"    

### A metaclass for bitfield

#### bitfields
* a bit field allows several integer values to be efficiently packed into a larger integer where each field uses a predefined number of bits
* bit-field width for dates
  + 31 days in one month need 5 bits to store (31).bit_length()
  + 12 months in each year need 4 bits to store
  + a year expressed in range of 1-9999 needs 14 bits
  + we can pack and store a date containing year, month, and day info to a 3-byte integer, wih one bit free
  + the computation for packing and unpacking will slow down the process quite a bit

In [66]:
# 31 days need 5 bits
print((31).bit_length())

# 12 months need 4 bits
print((4).bit_length())

# 9999 years need 14 bits
print((9999).bit_length())

5
3
14


#### Bitfield Tests
* write unit test for TDD
* first write unittest with class DateBitField without BitFieldMeta, and test_define_bitfield failed
* add BitFieldMeta class definition, and test passed
+ when test test_instantiate_bitfield_with_field_value, we failed since we didn't have init method to accept input 23
  + we modify the BitFieldMeta class by adding the dunder new method
  + by printing out the namesapce, we find the definition of day:5 is stored in an attribute named dunder annotations
    + we ceated a private attribute in namespace \_field\_widths and store the value of dunder annotations
  + to make the DateBitField class accept arguments by duder init, we create a new class BitFieldBase, and make it as a base clase of DateBitField class by adding it to bases in the BitFieldMeta to pass the test
  + remove all prints from the BitFieldMeta class
+ the next step is to think about what could go wrong such as
  + Bitfield contanis no fields
  + constructor argument names don't match field names
  + field width annotation is not an integer
  + field width is not positive
  + field name clashes with internals (internal variable names)
+ we starts with a test with no settings of day attribute and expect to see a TypeError, but surprisingly, we got an error that without setting the attribute caused the erase of the dunder annotation entry, so we changed the code in dunder new, to use get method of namespace, if if field_widths is None, raise the TypeError to pass the 'test_bitfield_without_fields_raises_type_error' test 
+ in 'test_mismatched_constructor_argument_names_raises_type_error', we delibrately input wrong field names to the constructor of DateBitField class, and expect to see the TypeError with information about which field nanmes are wrong from the exception
  + to do this, we modified dunder init method in BitFieldBase class. 
    + first, find the difference between the input keyword args and \_field\_widths from the namespace, which can be directly accessed by type(self).\_field\_widths (note that these attributes are not defined by dunder init of the dateBitField class, but is defined directly in the namespace of the DateBitField class. Although the code is in BitFieldBase class, the self object points to DateBitField class)
    + if the lenght of mismatched args >0, we raise the TypeError with error message
    + we put all these logic in a separate function \_validate\_arg\_names, and call it from dunder init of class BitFieldBase
* test non integer annotation value raises type error
  + instead of raise TypeError in BitFieldBase class, we test the field_width dictionary from dunder new metod of BitFieldMeta class
+ test negative field width raises type error: same as test non integer annotation value raises type error
+ test field name with leading underscore raises type error: prevent any private fields that may collide with internals

In [140]:
class BitFieldBase:
    
    def __init__(self, **kwargs):
        self._validate_arg_names(kwargs)
        self._validate_arg_values(kwargs)
        for field_name in type(self)._field_widths:
            setattr(self, field_name, 0)
            
        for key, value in kwargs.items():
            setattr(self, key, value)
        
    def _validate_arg_names(self, kwargs):    
        mismatched_args = set(kwargs).difference(type(self)._field_widths)
        if len(mismatched_args) !=0:
            raise TypeError(
                "{}.__init__() got unexpected keyword argument{}: {}".format(
                    type(self).__name__,
                    "" if len(mismatched_args) == 1 else "s",
                    ", ".join(
                        repr(arg_name) for arg_name in kwargs
                        if arg_name in mismatched_args
                    )
                )
            )
            
    def _validate_arg_values(self, kwargs):
        field_widths = type(self)._field_widths
        for key, value in kwargs.items():
            width = field_widths[key]
            min_value = 0
            max_value = 2 ** width -1
            
            if not (min_value <= value <= max_value):
                raise ValueError(
                    f"{type(self).__name__} field {key!r} "
                    f"got value {value!r} which is out of "
                    f"range {min_value}-{max_value} for a {width}-bit field"
                )
     
    def __int__(self):
        accumulator = 0
        shift = 0
        
        for name, width in type(self)._field_widths.items():
            value = getattr(self, name)
            accumulator |= value << shift
            shift += width
            
        return accumulator 
    
    def to_bytes(self):
        v = int(self)
        num_bytes = (sum(self._field_widths.values()) + 7) // 8
        return v.to_bytes(
            length=num_bytes,
            byteorder="little",
            signed=False,                         
        )
        
    
class BitFieldMeta(type):
    def __new__(mcs, name, bases, namespace, **kwargs):     
        
        field_widths = namespace.get("__annotations__", {})
        if len(field_widths) == 0:
            raise TypeError(
                f"{name} with metaclass {mcs.__name__} has no fields"
            )
        for field_name, width in field_widths.items():
            if not isinstance(width, int):
                raise TypeError(
                    f"{name} field {field_name!r} has annotation "
                    f"{width!r} that is not an integer"
                    
                )
                
            if width < 1:
                raise TypeError(
                    f"{name} field {field_name!r} has non-positive "
                    f"field width {width}"                
                )
                
            if field_name.startswith("_"):
                raise TypeError(
                    f"{name} field {field_name!r} begins with an underscore"
                )
        
        namespace["_field_widths"] = field_widths
        bases = (BitFieldBase,) + bases
        return super().__new__(mcs, name, bases, namespace)        

In [141]:
import unittest

class TestBitField(unittest.TestCase):
    """Test BItFieldMeta and the bitfield claases it makes"""
    
    def test_define_bitfield(self):
        
        class DateBitField(metaclass=BitFieldMeta):
            day: 5

    def test_instantiate_default_bitfield(self):
        class DateBitField(metaclass=BitFieldMeta):
            day: 5
        
        _ = DateBitField()
        
    def test_instantiate_bitfield_with_field_value(self):
        class DateBitField(metaclass=BitFieldMeta):
            day: 5
        
        _ = DateBitField(day=23)
     
    def test_bitfield_without_fields_raises_type_error(self):
        
        with self.assertRaises(TypeError):
            
            class EmptyBItField(metaclass=BitFieldMeta):
                pass
   
    def test_mismatched_constructor_argument_names_raises_type_error(self):
        
        class DateBitField(metaclass=BitFieldMeta):
            day: 5
            month: 4
            year: 14
                
        with self.assertRaises(TypeError) as exc_info:
            _= DateBitField(day=13, mnth=5, yr=1999)
            
        self.assertEqual(
            str(exc_info.exception),
            "DateBitField.__init__() got unexpected keyword arguments: "
            "'mnth', 'yr'"
        )
        
    def test_non_integer_annotation_value_raises_type_error(self):
        
        with self.assertRaises(TypeError) as exc_info:
            
            class DateBitField(metaclass=BitFieldMeta):
                day: "Wednesday"
                    
        self.assertEqual(
            str(exc_info.exception),
            "DateBitField field 'day' has annotation 'Wednesday' "
            "that is not an integer"
        )
        
    def test_zero_field_width_raises_type_error(self):
        with self.assertRaises(TypeError) as exc_info:
            
            class DateBitField(metaclass=BitFieldMeta):
                day: 0
                    
        self.assertEqual(
            str(exc_info.exception),
            "DateBitField field 'day' has non-positive field width 0"
        )            
        
    def test_non_negative_annotation_value_raises_type_error(self):

        with self.assertRaises(TypeError) as exc_info:

            class DateBitField(metaclass=BitFieldMeta):
                day: -1

        self.assertEqual(
            str(exc_info.exception),
            "DateBitField field 'day' has non-positive field width -1"            
    )

    def test_field_name_with_leading_underscore_raises_type_error(self):
        
        with self.assertRaises(TypeError) as exc_info:

            class DateBitField(metaclass=BitFieldMeta):
                _day: 5

        self.assertEqual(
            str(exc_info.exception),
            "DateBitField field '_day' begins with an underscore"
        )            
                
    def test_initialization_out_of_lower_field_range_raises_value_error(self):
        
        class DateBitField(metaclass=BitFieldMeta):
            day: 5
                
        with self.assertRaises(ValueError) as exc_info:
            _ = DateBitField(day=-1)
            
        self.assertEqual(
            str(exc_info.exception),
            "DateBitField field 'day' got value -1 "
            "which is out of range 0-31 for a 5-bit field"
        ) 
        
        
    def test_initialization_out_of_upper_field_range_raises_value_error(self):
        
        class DateBitField(metaclass=BitFieldMeta):
            day: 5
                
        with self.assertRaises(ValueError) as exc_info:
            _ = DateBitField(day=32)
            
        self.assertEqual(
            str(exc_info.exception),
            "DateBitField field 'day' got value 32 "
            "which is out of range 0-31 for a 5-bit field"
        )
        
    def test_fields_are_default_initialized_to_zero(self):
        class DateBitField(metaclass=BitFieldMeta):
            day: 5
        
        d = DateBitField()
        print(d.__dict__)
        
        self.assertEqual(d.day, 0)
        
        
    def test_initialized_field_values_can_be_retrieved(self):
        class DateBitField(metaclass=BitFieldMeta):
            day: 5
        
        d = DateBitField(day=17)
                
        self.assertEqual(d.day, 17)
    
    def test_conversion_to_integer(self):
        
        class DateBitField(metaclass=BitFieldMeta):
            day: 5
            month: 4
            year: 14
                
        d = DateBitField(day=25, month=3, year=2010)
        i = int(d)
        self.assertEqual(
        i,
        0b00011111011010_0011_11001
        # <--------2010> <-3> <-25>  
        )

    def test_conversion_to_bytes(self):

        class DateBitField(metaclass=BitFieldMeta):
            day: 5
            month: 4
            year: 14

        d = DateBitField(day=25, month=3, year=2010)
        b = d.to_bytes()
        self.assertEqual(
            b,
            (0b00011111011010_0011_11001).to_bytes(3, "little", signed=False)
        )
            
# run unittest
unittest.main(argv=[''], verbosity=2, exit=False)    

test_bitfield_without_fields_raises_type_error (__main__.TestBitField) ... ok
test_conversion_to_bytes (__main__.TestBitField) ... ok
test_conversion_to_integer (__main__.TestBitField) ... ok
test_define_bitfield (__main__.TestBitField) ... ok
test_field_name_with_leading_underscore_raises_type_error (__main__.TestBitField) ... ok
test_fields_are_default_initialized_to_zero (__main__.TestBitField) ... ok
test_initialization_out_of_lower_field_range_raises_value_error (__main__.TestBitField) ... ok
test_initialization_out_of_upper_field_range_raises_value_error (__main__.TestBitField) ... ok
test_initialized_field_values_can_be_retrieved (__main__.TestBitField) ... ok
test_instantiate_bitfield_with_field_value (__main__.TestBitField) ... ok
test_instantiate_default_bitfield (__main__.TestBitField) ... ok
test_mismatched_constructor_argument_names_raises_type_error (__main__.TestBitField) ... ok
test_non_integer_annotation_value_raises_type_error (__main__.TestBitField) ... ok
test_non_n

{'day': 0}
